# CartPole
CartPole ist ein Spiel das von OpenAI im gym-Paket zur Verfügung gestellt wird. <br/>
Dabei handelt es sich um ein inverses Pendel, welches mit rechts-links-Bewegungen balanciert werden muss.<br/>
In dieser Lösung des Problems werden zuerst über zufällige Aktionen Testdaten generiert.<br/>
Mit den aufbereiteten Testdaten wird dann ein Neuronales Netz trainiert. Hierzu wird tflearn verwendet.<br/>
Abschließend wird der Lernerfolg des Neuronalen Netzes gemessen und das Model wird gespeichert.<br/>

## Setup
Im Setup werden die nötigen Bibliotheken importiert und die festen Variablen inizialisiert. <br/>
Über die Variablen kann die Performance des Neuronalen Netzes beeinflusst werden.

In [ ]:
# Imports

import gym
import random
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter
import os
import sys
import webbrowser
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import trange

In [ ]:
# Defining the static variables

LR = 1e-3
global max_score, VISIBLE
max_score = 10000
goal_steps = 10000
score_requirement = 110
initial_games = 1000
VISIBLE = False

# Register an own game instanz
gym.envs.register(
    id='MyCartPole-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    tags={'wrapper_config.TimeLimit.max_episode_steps' : max_score},
)

env = gym.make("MyCartPole-v0")

## Funktionen definieren
Um einen einfachen und objektorientierten Ansatz zu haben, wurden alle Aktionen in eine eigene Funktion gepackt.

Die some_random_games Funktion wurde erstellt, um das Verhalten des Spiels und der Umgebung zu analysieren.<br/>
So wurden der Output von env.step(action) und die Funktion von enc.action_space.sample() ersichtlich.

In [ ]:
# Function to show how a random game works with the OpenAI gym
# Input: None
# Output: None
# Shows a game in a seperate window

def some_random_games():
    action_min = 100
    action_max = -1
    score = 0
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            #action = random.randrange(0,6)
            print("Action","\n",action)
            observation, reward, done, info = env.step(action)
            print("Observation","\n", "position of cart, velocity of cart, angle of pole, rotation rate of pole", "\n",observation) 
            print("Reward","\n",reward)
            print("Done","\n",done)
            print("Info","\n",info)
            
            score += reward
            if(action < action_min):
                action_min = action
            if(action > action_max):
                action_max = action
            
            if(done):
                print("Actions: ", action_min, " -> ", action_max)
                print("Score: ", score)
                break

Die determine_training_action ist eine Funktion, die <a href="https://github.com/jeff-collins">Jeff Collins</a> für seine Lösung des CartPole Problems programmiert und auf <a href="https://gym.openai.com/evaluations/eval_iyQVt3aT9yqyIgw2RBFug">www.gmy.openai.com</a> zur Verfügung gestellt hat. <br/>
Diese Funktion sagt mit einer hohen Wahrscheinlichkeit die richtige nächste Aktion vorraus, wodurch bessere Trainigsdaten erstellt werden können.

In [ ]:
# Function to predict the next action
# Input: observation - list with 4 floats
# Output: action - inf (0 or 1)
# Predicts on a simple, static way the next action

def determine_training_action(observation):
    if (observation[3] > 0): 
        action = 1
    else:
        action = 0


    if (abs(observation[3]) < .2):
        xmodifier = 0;

        if (abs(observation[0]) > .2):
            if(observation[0] > 0):
                xmodifier = -.005
            else:
                xmodifier = .005

        if (abs(observation[1]) > .1):
            if (observation[1] > 0):
                xmodifier += -.1
            else:
                xmodifier += .1

        if (observation[2] > xmodifier):
            action = 1
        else:
            action = 0

        if (observation[0] > .2 and observation[2] < .1 and observation[2] >= 0):
            action = 1

        if (observation[0] < -.2 and observation[2] > -.1 and observation[2] <= 0):
            action = 0

    return action

Mit den Aktionen aus der determine_training_action Funktion werden nun die Trainingsdaten erstellt. <br/>
Die Funktion gibt eine Liste mit Trainingsdaten zurück. Außerdem werden die Anzahl der falschen und richtigen Entscheidungen, sowie die durchschnittliche Punktzahl ausgegeben.

In [ ]:
# Function to generate the inital dataset
# Input: None
# Output: train_data - list with two arrays
# Playes not rendered games. The number is defined by *inital_games*. Shows a progressbar.

def initial_population():    

    train_data = []
    scores = []
    accepted_scores = []
    optimal = [0,0,0,0]
    error = 0
    correct = 0

    for index, _ in tqdm(enumerate(range(initial_games)), total=initial_games):
        score = 0
        game_memory = []
        prev_observation = env.reset()
        
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            observation, reward, done, info = env.step(action)

            if len(prev_observation) > 0:
                if determine_training_action(prev_observation) == action:
                    game_memory.append([prev_observation,action])
                    error += 1
                else:
                    correct +=1
                    
                
            prev_observation = observation
            score += reward
            if done:
                break
                
        for data in game_memory:
            if data[1] == 1:
                output = [0,1]
            elif data[1] == 0:
                output = [1,0]
            train_data.append([data[0], output])
                

                
        env.reset()
        scores.append(score)
    
    action_sum = error + correct
    
    print("Error: ", error, "(", ((error * 100) // action_sum) ,"%) - Correct: ", correct, "(",((correct * 100) // action_sum), "%)")
    print('Average score: ', mean(scores))
    print('Median core: ', median(scores))
    print(Counter(scores))
    
    return train_data
     

Die Funktion neural_network_model erstellt ein Neuronales Netz mit vier Eingabeneuronen und zwei Ausgabeneuronen. Dabei werden fünf Hiddenlayer genutzt. Um ein Auswendiglernen des Netzes zu verhindern, wird ein Dropout von 0.8 auf jedem Hiddenlayer durchgeführt. Da eine eindeutige Ausgabe notwendig ist, wird softmay verwendet. <br/>
Die Regression wird mit dem Optimizer <a href="http://tflearn.org/optimizers/">Adam</a> und der Loss-Funktion <a href="http://tflearn.org/objectives/">Categorical Crossentropy</a> ausgeführt.<br/>
Adam ist eine Methode der stochastischen Optimierung.<br/>
Categorical Crossentropy, im deutschen kategorische Kreuzentropie, nutzt die Grundlagen der <a href="https://de.wikipedia.org/wiki/Kreuzentropie">Kreuzentropie</a>, um den Wahrscheinlichkeitsfehler in diskreten Klassifizierungsaufgaben, bei denen sich die Klassen gegenseitig ausschließen, zu messen.


In [ ]:
# Funktion to generate a neural network model
# Input: input_size
# Output: neuronal network model
# Generates the neuronal network.

def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(network,tensorboard_verbose=1, tensorboard_dir='log/tensorboard')
    
    return model

Um das Neuronale Netz wird mit den erstellten Trainingsdaten trainiert. Dabei werden mehrere Epochen durchlaufen. 

In [ ]:
# Funktion to prepare and train the data
# Input: train_data
#        epochen (optional)
#        steps (optional)
#        model (optional)
# Output: model
# Prepares the data and trains the neuronal network with this data.

def train_model(train_data, epochen=1,steps=10, model=False):
    x = np.array([i[0] for i in train_data]).reshape(-1,len(train_data[0][0]), 1)
    y = [i[1] for i in train_data]
    
    if not model:
        model = neural_network_model(input_size= len(x[0]))
        
    model.fit({'input':x}, {'targets':y}, n_epoch=epochen, snapshot_step=steps, show_metric=True)
    
    return model

Um das Neuronale Netz zu testen muss das Spiel gespielt werden. Hier gibt es die Möglichkeit dies ohne ein Rendering auszuführen, was eine schnellere Ausführung möglich macht. Bei einem perfekten Neuronalen Netz dauert eine Auführung max_score / 1000 Sekunden, also 10 Sekunden. Daher sollte die Anzahl der Spiele mit bedacht gewählt werden. <br/>
Die Funktion gibt die durchschnittliche Punktzahl zurück und gibt den Score und die Verteilung der Entscheidungen aus. <br/>
Das Problem gilt laut <a href="https://gym.openai.com/envs/CartPole-v0">www.openai.com</a> als gelöst, wenn der durschnittliche Score bei über 195 Punkten liegt. 

In [ ]:
# Funktion to play a game
# Input: model
#        visibility (optional)
# Output: avarege_score
# Plays *in_number* games to show the progress of the model. Also shows a progressbar if the games are not rendered.

def play_game(in_model, in_number=100, visibility=False):
    scores = []
    choices = []
    optimize_data = []

    for each_game in tqdm(range(in_number), total=in_number):
        score = 0
        game_memory = []
        prev_obs = env.reset()
        for _ in range(goal_steps):
            if (VISIBLE or visibility):
                env.render()
                
            if len(prev_obs) == 0:
                action = random.randrange(0,2)
            else:
                action = np.argmax(in_model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
                
            choices.append(action)
            new_obs, reward, done, info = env.step(action)
            prev_obs = new_obs
            game_memory.append([new_obs, action])
            score += reward
            
            if done:
                break
        
        if score > 300:
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                optimize_data.append([data[0], output])
        
        scores.append(score)
    
    average_score = sum(scores)/len(scores)
    print('Scores: ',scores)
    print('Average Score ', average_score)
    print('Choice 0: {}, Choice 1: {}'.format(choices.count(0)/len(choices), choices.count(1)/len(choices)))
    return average_score


## Trainingsdaten erstellen
Über die Funktion werden Trainingsdaten erstellt.

In [ ]:
# Generate data
training_data = initial_population()

## Model trainieren
Es wird ein Neuronales Netzwerk erstellt und als Model gespeichert. Dieses Model wird dann mit den Trainingsdaten trainiert. <br/>
Je nach Größe des Trainingsdatensets kann das etwas dauern.

In [ ]:
# Create and train model
model = train_model(training_data)

## Testspiel
Im Testspiel wird ohne Rendering der durschnittliche Score des Models ermittelt und ausgegeben. <br/> Bei 100 Spielen und bis zu 30 Sekunden pro Spiel, kann dieser Vorgang 45 Minuten dauern.

In [ ]:
# Test the model by playing the game.
modelscore = play_game(model, 100)

In [ ]:
print("Model Score: ",modelscore)

## Model speichern
Um das Model später erneut laden zu können oder weitere Optimierungen durchführen zu können, wird das Model gespeichert.

In [ ]:
# Save the model in a folder
modelscore = int(round(modelscore))
if(modelscore > 200):
    path = "models/" + str(modelscore) + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    model.save(path + str(modelscore) + ".model")

## Finales Spiel
Im finalen Spiel werden die Spiele gerendert.

In [ ]:
# Show the user the game
play_game(model, 10, True)

## Kurven
Die Kurven können über das Tensorboard abgerufen werden. Hierzu kann 'tensorboard --logdir log/tensorboard/' in die Console eingegeben werden.<br/> Der Nachfolgende Codeblock startet das TensorBoard.<br/> Sollte die Seite nicht geladen werden nach 10 Sekunden refreshen.

In [ ]:
webbrowser.open('http://127.0.1.1:6006')

!tensorboard --logdir="log/tensorboard"